### Import data

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib as map

In [3]:
from pathlib import Path

# Path.cwd() gives you the *current working directory* (where the notebook is).
# Because the notebook lives in "notebooks/", we go one level up with .parent
# that lands us in the project root folder: hrv-readiness-study/
ROOT = Path.cwd().parent  

# Build the path to the "data/raw" folder inside the project root.
# This is equivalent to writing "../data/raw" if you're in notebooks/,
# but much clearer and more stable.
RAW = ROOT / "data" / "raw"  

# Read the CSV file stored in data/raw. 
# RAW / "wellness.csv" joins the folder path with the filename safely,
# so you don’t have to worry about slashes on different OSes.
hrv = pd.read_csv(RAW / "wellness.csv")

hrv.head(2)


,Timestamp,Today's Date,How did you feel today?,Whoop recovery score,Resting RHR,Calories,Whoop sleep score %,Whoop sleep hours,Trained?,Any notes?
0,9/26/2025 11:10:28,9/26/2025,2,Green,62,Deficit,85,6:15,Yes,Felt somewhat sick all day. Very drained. Low ...


In [4]:
# begin cleaning the data

#rename columns
hrv_cleaned = hrv.rename(columns={"Timestamp": "timestamp", "Today's Date": "date",
    "How did you feel today?": "feeling",
    "Whoop recovery score": "whoop_status",
    "Resting RHR": "rhr",
    "Calories": "calories",
    "Whoop sleep score %": "sleep_score",
    "Whoop sleep hours": "sleep_hhmm",
    "Trained?": "trained",
    "Any notes?": "notes",})

hrv_cleaned

,timestamp,date,feeling,whoop_status,rhr,calories,sleep_score,sleep_hhmm,trained,notes
0,9/26/2025 11:10:28,9/26/2025,2,Green,62,Deficit,85,6:15,Yes,Felt somewhat sick all day. Very drained. Low ...


In [5]:
hrv_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   timestamp     1 non-null      object
 1   date          1 non-null      object
 2   feeling       1 non-null      int64 
 3   whoop_status  1 non-null      object
 4   rhr           1 non-null      int64 
 5   calories      1 non-null      object
 6   sleep_score   1 non-null      int64 
 7   sleep_hhmm    1 non-null      object
 8   trained       1 non-null      object
 9   notes         1 non-null      object
dtypes: int64(3), object(7)
memory usage: 212.0+ bytes


In [9]:
# turn date into datetime 
hrv_cleaned["date"] = pd.to_datetime(hrv_cleaned["date"], errors="raise")

# check output 
hrv_cleaned["date"].dtype


dtype('<M8[ns]')

In [12]:
# extract year and date
hrv_cleaned["year"] = hrv_cleaned["date"].dt.year
hrv_cleaned["month"] = hrv_cleaned["date"].dt.month
hrv_cleaned["day"] = hrv_cleaned["date"].dt.day

# filter by ranges
hrv_cleaned[hrv_cleaned["date"] == "26-09-25"]


,timestamp,date,feeling,whoop_status,rhr,calories,sleep_score,sleep_hhmm,trained,notes,year,month,day
0,9/26/2025 11:10:28,2025-09-26,2,Green,62,Deficit,85,6:15,Yes,Felt somewhat sick all day. Very drained. Low ...,2025,9,26
